In [100]:
import pandas as pd
import matplotlib as plt
import numpy as np
import matplotlib.widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, interactive_output, fixed, FloatSlider, Layout
import matplotlib.pyplot as plt
import math
import sys
import scipy.special
from scipy.special import erf
from scipy.stats import chi2, poisson
import warnings

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [101]:
class DiscreteSamplingDistribution:
    def __init__(self, df):
        self.df = df
        self.n = self.df.to_numpy().sum(axis=0)
        self.size = self.df.shape[0]

    def show_database(self):
        pd.set_option('display.max_rows', self.df.shape[0]+1)
        styler = self.df.style.set_properties(**{'background-color': '#B0C4DE', 'color': '#154360'})
        display(styler) 

    def get_mean(self):
        mean = np.sum(self.df.index.to_numpy() * self.df["ni"].to_numpy())/self.n  
        return mean

    def show_initial_database(self):
        styler = self.df[["ni"]].style.set_properties(**{'background-color': '#B0C4DE', 'color': '#154360'})
        display(styler)

    @staticmethod
    def show_interval_database(df):
        styler = df[["ni", "pi"]].style.set_properties(**{'background-color': '#B0C4DE', 'color': '#154360'})
        display(styler) 
    
    def get_deviation(self):
        mean = self.get_mean()
        norm = self.df.index.to_numpy() - mean
        norm_square = np.square(norm) * self.df.ni
        return np.sum(norm_square)

    def get_dispersion(self):
        return self.get_deviation() / self.n

    def get_standard_error(self):
        return math.sqrt(self.get_dispersion())

    def Binomial_distribution_probabilitis(self):
        N = self.df.index[-1]
        if (self.p == -1) : self.p = self.get_mean()/N
        C = scipy.special.factorial(N)/ (scipy.special.factorial(self.df.index) * scipy.special.factorial(N - self.df.index))
        P = C * np.power(self.p, self.df.index) * np.power((1 - self.p), N - self.df.index)
        self.df["pi"] = P

    # def Poisson_distribution_probabilitis(self):
    #     l = self.get_mean()
    #     P = np.power(l, self.df.index)*np.exp(-l)/scipy.special.factorial(self.df.index)
    #     P[-1] = 1 - np.sum(P[:-1])
    #     self.df["pi"] = P

    def normalise(self): 
        p = self.df.pi.to_numpy()
        ni = self.df.ni.to_numpy() 
        n = self.size
        xi = self.df.index.to_numpy().astype(str)
        
        i = 0
        while (i < n - 1):
            if (p[i] * self.n < 10) or ni[i] < 5:
                p[i] += p[i + 1]
                ni[i] += ni[i + 1]
                xi[i] += " " + xi[i + 1]
                xi = np.delete(xi, i + 1)
                p = np.delete(p, i + 1)
                ni = np.delete(ni, i + 1)
                n -= 1
                i -= 1
            i +=1
        if p[n-1] * self.n < 10 or ni[n-1] < 5:
            p[n-2] += p[n-1]
            ni[n-2] += ni[n-1]
            xi[n-2] += " " + xi[n-1]
            xi = np.delete(xi, n-1)
            p = np.delete(p, n-1)
            ni = np.delete(ni, n-1)
            n -= 1
            i -= 1

        self.df_joined = pd.DataFrame({'ni': ni, 'pi': p}, xi)
        self.size_joined = self.df_joined.shape[0]
        # self.df["midd"] = ((self.df.x0.to_numpy() + self.df.x1.to_numpy()) / 2).astype(float)

    def Pearson_cumulative_test_statistic(self):
        hi = np.sum(np.power(self.df_joined.ni - self.n * self.df_joined.pi, 2)/(self.n * self.df_joined.pi))
        return hi

    def Pearson_cumulative_test_critical_val(self, s, a=0.05):  
        df = self.size_joined - 1 - s
        chi = chi2.isf(q=a, df=df)
        return chi

    def print_results(self, a, pi=-1):
        self.p = pi
        am = 1 if (self.p == -1) else 0
        print("\nГІПОТЕЗА Н0 - Біномний розподіл.")
        self.Binomial_distribution_probabilitis()
        self.show_interval_database(self.df)
        print("\nСТАТИСТИЧНІ ДАНІ ПІСЛЯ ПЕРЕВІРКИ УМОВ НОРМУВАННЯ: ")
        self.normalise()
        self.show_interval_database(self.df_joined)
        emp = self.Pearson_cumulative_test_statistic()
        crit = self.Pearson_cumulative_test_critical_val(am, a)
        print("\nЧИСЛОВІ ХАРАКТЕРИСТИКИ: ")
        print("\n* p: ", self.p)
        print("\n* Рівень значущості: ", a)
        print("\n* Число часткових інтервалів: ", self.size_joined)
        print("\n* Число параметрів густини гіпотетичного розподілу: ", am)
        print("\n* Число ступенів вільності: ", self.size_joined - 1 - am)
        print("\n* ЕМПІРИЧНЕ значення критерію узгодження Пірсона: ", emp)
        print("\n* КРИТИЧНЕ значення критерію узгодження Пірсона: ", crit)
        print("\nВИСНОВОК: H0(Біномний розподіл) - ", "ПРИЙМАЄМО" if emp < crit else "ВІДХИЛЯЄМО")


    @staticmethod
    def draw_plot(x_label, y_label, title):
        fig, ax = plt.subplots(figsize=(10, 7))
        ax.set_title(title, color='black', size=25, fontweight='bold', pad=30)
        ax.set_ylabel(y_label, size=15, fontweight='bold')
        ax.set_xlabel(x_label, size=15, fontweight='bold')
        ax.patch.set_facecolor('#B0C4DE')
        fig.patch.set_facecolor('#154360')
        plt.grid(zorder=0)

    def counts_diagram(self):
        DiscreteSamplingDistribution.draw_plot("x", "n", "Діаграма Частот")
        plt.bar(self.df.index, self.df['ni'], color="#CD5C5C", zorder=3, edgecolor='red')
        plt.legend(['Частота'], loc='upper left')
        plt.xticks(self.df.index)
        plt.yticks(self.df['ni'])
        plt.show()
    
    def counts_polygon(self):
        DiscreteSamplingDistribution.draw_plot("x", "n", "Полігон Частот")
        plt.plot(self.df.index, self.df['ni'], '-ok', color="red")
        plt.legend(['Частота'], loc='upper left')
        plt.xticks(self.df.index)
        plt.yticks(self.df['ni'])
        plt.show()


In [102]:
class IntervalSamplingDistribution(DiscreteSamplingDistribution):
    def __init__(self, df):
        super().__init__(df)
        interval_x = np.array([np.array(xi.split('-')).astype(float) for xi in self.df.index])
        self.df["x0"] = interval_x[:, 0]
        self.df["x1"] = interval_x[:, 1]  
        self.df["midd"] = ((self.df.x0.to_numpy() + self.df.x1.to_numpy()) / 2).astype(float)

    def get_mean(self):
        mean = np.sum(self.df["midd"].to_numpy() * self.df["ni"].to_numpy())/self.n  
        return mean
    
    def get_deviation(self):
        mean = self.get_mean()
        norm = self.df["midd"].to_numpy() - mean
        norm_square = np.square(norm) * self.df.ni
        return np.sum(norm_square)

    def get_dispersion(self):
        return self.get_deviation() / self.n

    def get_standard_error(self):
        return math.sqrt(self.get_dispersion())  

    def count_probabilitis(self):
        self.df.x0.iloc[0] = -math.inf
        self.df.x1.iloc[self.size - 1] = math.inf
        if self.a == -1: self.a = self.get_mean()
        if self.s == -1: self.s = self.get_standard_error()
        Phi = lambda x: erf(x/2**0.5)/2
        self.df["pi"] = Phi((self.df.x1 - self.a)/self.s) - Phi((self.df.x0 - self.a)/self.s)

    def normalise(self): 
        p = self.df.pi.to_numpy()
        ni = self.df.ni.to_numpy()
        x0 = self.df.x0.to_numpy()
        x1 = self.df.x1.to_numpy()
        
        n = self.size
        i = 0
        while (i < n - 1):
            if (p[i] * self.n < 10) or ni[i] < 5:
                p[i] += p[i + 1]
                ni[i] += ni[i + 1]
                x0[i] = min(x0[i], x0[i + 1])
                x1[i] = max(x1[i], x1[i + 1])
                p = np.delete(p, i + 1)
                ni = np.delete(ni, i + 1)
                x0 = np.delete(x0, i + 1)
                x1 = np.delete(x1, i + 1)
                i -= 1
                n -= 1
            i += 1

        if p[n-1] * self.n < 10 or ni[n-1] < 5:
            p[n-2] += p[n-1]
            ni[n-2] += ni[n-1]
            x0[n-2] = min(x0[n-1], x0[n-2])
            x1[n-2] = max(x1[n-1], x1[n-2])
            p = np.delete(p, n-1)
            ni = np.delete(ni, n-1)
            x0 = np.delete(x0, n-1)
            x1 = np.delete(x1, n-1)
            n -= 1
            i -= 1
        
        
        intervals = np.stack((x0, x1), axis=1)
        self.df_joined = pd.DataFrame({'ni': ni, 'pi': p, 'x0': x0, 'x1': x1}, self.interval_x_toString(intervals))
        self.size_joined = self.df_joined.shape[0]
        self.df_joined["midd"] = ((self.df_joined.x0.to_numpy() + self.df_joined.x1.to_numpy()) / 2).astype(float)

    def Pearson_cumulative_test_statistic(self):
        hi = np.sum(np.power(self.df_joined.ni - self.n * self.df_joined.pi, 2)/(self.n * self.df_joined.pi))
        return hi

    def Pearson_cumulative_test_critical_val(self, s,  a=0.05):  
        df = self.size_joined - 1 - s
        chi = chi2.isf(q=a, df=df)
        return chi

    def print_results(self, alpha, a, s):
        self.a = a
        self.s = s
        print("\nГІПОТЕЗА Н0 - Нормальний розподіл.")
        am = 0
        if (self.a == -1): am += 1
        if (self.s == -1): am += 1
        self.count_probabilitis()
        self.show_interval_database(self.df)
        print("\nСТАТИСТИЧНІ ДАНІ ПІСЛЯ ПЕРЕВІРКИ УМОВ НОРМУВАННЯ: ")
        self.normalise()
        self.show_interval_database(self.df_joined)
        emp = self.Pearson_cumulative_test_statistic()
        crit = self.Pearson_cumulative_test_critical_val(am, alpha)
        print("\nЧИСЛОВІ ХАРАКТЕРИСТИКИ: ")
        print("\n* a: ", self.a)
        print("\n* s: ", self.s)
        print("\n* Рівень значущості: ", alpha)
        print("\n* Число часткових інтервалів в інтервальному варіаційному ряді: ", self.size_joined)
        print("\n* Число параметрів густини гіпотетичного розподілу: ", am)
        print("\n* Число ступенів вільності: ", self.size_joined - 1 - am)
        print("\n* ЕМПІРИЧНЕ значення критерію узгодження Пірсона: ", emp)
        print("\n* КРИТИЧНЕ значення критерію узгодження Пірсона: ", crit)
        print("\nВИСНОВОК: H0(Нормальний розподіл) - ", "ПРИЙМАЄМО" if emp < crit else "ВІДХИЛЯЄМО")

    def draw_histogram(self):
        h = self.df.ni/(self.df.x1-self.df.x0)
        DiscreteSamplingDistribution.draw_plot("x", "h", "Гістограма Частот")
        k = np.append(np.array([self.df.x0[0], self.df.x1[0]]), self.df.x1[1:])
        s = k + 0.0001

        plt.hist(s[:-1], bins=k, weights=h, color='#5499C7', edgecolor='#154360', zorder=3)
        plt.xticks(k)
        plt.yticks(h)
        plt.show()
    
    @staticmethod
    def interval_x_toString(interval_x):
        res = np.array([f"({round(x[0], 2)}, {round(x[1], 2)}]" for x in interval_x])
        res[0] = f"[{round(interval_x[0][0], 2)}, {round(interval_x[0][1], 2)}]"
        return res

In [103]:

pi1 = widgets.FloatText(value=-1, description='P: ', layout=Layout(width='20%', height='30px'))
a1 = widgets.FloatText(value=-1, description='a: ', layout=Layout(width='20%', height='30px'))
s1 = widgets.FloatText(value=-1, description='s: ', layout=Layout(width='20%', height='30px'))
alpha1 = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.001, description='Рівень значущості: ', layout=Layout(width='40%', height='30px'), style={'description_width': 'initial'})
alpha2 = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.001, description='Рівень значущості: ', layout=Layout(width='40%', height='30px'), style={'description_width': 'initial'})
t2 = widgets.ToggleButton(value=False, description='Гіпотеза про Біномний розподіл', button_style='info', icon='check', layout=Layout(width='50%', height='35px'))
t3 = widgets.ToggleButton(description='Гіпотеза про нормальний розподіл', button_style='info', icon='check', layout=Layout(width='50%', height='35px'))
out1 = widgets.Output(layout=widgets.Layout(border = '1px solid black'))
out2 = widgets.Output(layout=widgets.Layout(border = '1px solid black'))
out3 = widgets.Output(layout=widgets.Layout(border = '1px solid black'))
out4 = widgets.Output(layout=widgets.Layout(border = '1px solid black'))

In [104]:
def results(dist, a, pi):
    @interact(counts_polygon=False, counts_diagram=False, res=False)
    def f(counts_polygon, counts_diagram, res):
        if (counts_polygon):
            dist.counts_polygon()
        if (counts_diagram):
            dist.counts_diagram()
        if (res):
            if (pi == -1):
                dist.print_results(a)
            else:
                dist.print_results(a, pi)
            

In [105]:
def results_interval(dist, alpha, a, s):
    @interact(counts_histogram=False, res=False)
    def f(counts_histogram, res):
        if (counts_histogram):
            dist.draw_histogram()
        if (res):
            dist.print_results(alpha, a, s)

In [106]:
def part1():
    @interact(toggle2=t2)
    def function4(toggle2):
        with out1:
            if toggle2==True: 
                dff = pd.read_csv("task9.csv")
                var = widgets.Dropdown(value=18, options=np.unique(dff["devices"]))
                @interact(var = var, alpha1=alpha1, pi1=pi1)
                def f1(var, alpha1, pi1):
                    with out4:
                        df1 = dff[dff["devices"] == var]
                        df1 = df1.set_index("devices")
                        df1 = df1.rename(index={var: 'ni'})
                        df1 = df1.T
                        df1 = df1.set_index(df1.index.astype(float)) 
                        distribution = DiscreteSamplingDistribution(df1.copy())
                        out4.clear_output()
                        print("\nСТАТИСТИЧНІ ДАНІ  (дискретний статистичний розподіл): ")
                        distribution.show_initial_database()
                        results(distribution, alpha1, pi1)
        
            else:
                out1.clear_output()
                out4.clear_output()


In [107]:
def part2():

    @interact(toggle3=t3)
    def function4(toggle3):
        with out2:
            if toggle3==True: 
                df = pd.read_csv("task4.csv")
                var = widgets.Dropdown(value=18, options=np.unique(df["T"])) 
            
                @interact(var = var, alpha2=alpha2, a1=a1, s1=s1)
                def f4(var, alpha2, a1, s1):
                    with out3:
                        df2 = df[df["T"] == var]
                        df2 = df2.set_index("T")
                        df2 = df2.rename(index={var: 'ni'})
                        df2 = df2.T
                        distribution = IntervalSamplingDistribution(df2)
                        out3.clear_output()
                        print("\nСТАТИСТИЧНІ ДАНІ (Інтервальний статистичний розподіл): ")
                        distribution.show_initial_database()
                        results_interval(distribution, alpha2, a1, s1)

            else:
                out2.clear_output()
                out3.clear_output()

In [108]:
part2()
part1()
display(out1) 
display(out4) 
display(out2) 
display(out3) 

interactive(children=(ToggleButton(value=False, button_style='info', description='Гіпотеза про нормальний розп…

interactive(children=(ToggleButton(value=False, button_style='info', description='Гіпотеза про Біномний розпод…

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))